In [0]:
from pyspark.sql.functions import trim, when, sum

df = spark.read.table('`bike-data-lakehouse-workspace`.bronze.crm_prd_info')

In [0]:
df = df.withColumn('prd_nm', trim(df.prd_nm))

df = df.withColumn('prd_line', when(df.prd_line == 'R', 'Road').when(df.prd_line == 'M', 'Mountain').when(df.prd_line == 'S', 'Sport').when(df.prd_line == 'T', 'Touring').otherwise('Unknown'))

null_count = df.select(
    sum(when(df.prd_cost.isNull(), 1).otherwise(0)).alias("null_count")
).collect()[0]["null_count"]
                
df = df.filter(df.prd_cost.isNotNull())

print(df.count())
df = df.distinct()
print(df.count())

In [0]:
df = (
    df.withColumnRenamed('prd_id', 'Product_id')
    .withColumnRenamed('prd_key', 'Product_Key')
    .withColumnRenamed('prd_nm', 'Product_Name')
    .withColumnRenamed('prd_cost', 'Product_Cost')
    .withColumnRenamed('prd_line', 'Product_Line')
    .withColumnRenamed('prd_start_dt', 'Product_Start_Date')
    .withColumnRenamed('prd_end_dt', 'Product_End_Date')
)

df.display()

In [0]:
df.write.mode('overwrite').saveAsTable('`bike-data-lakehouse-workspace`.silver.crm_product_info')